In [1]:
#import libraries
import pandas as pd
import numpy as np
from datetime import datetime

timestamp= datetime.utcnow().strftime('%s')

#read csv files
df1= pd.read_csv(r"../dataset1.csv", )
df2= pd.read_csv(r"../dataset2.csv")

#combine into one big df
df3= df1.append(df2)
df3.reset_index(inplace=True, drop=True)

df1.head()

<ipython-input-1-68e716223b7f>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3= df1.append(df2)


,name,price
0,William Dixon,109.037280
1,Kristen Horn,262.524652
2,Kimberly Chang,187.007258
3,Mary Ball,283.174648
4,Benjamin Craig,143.871582


## Requirement 1: Delete any rows which do not have a name

In [2]:
df3.dropna(subset=['name'], inplace=True)

## Requirement 2: Split the name field into first_name, and last_name

In [3]:
#UDF to split name into first and last name
def split_name(name_col):
    
    not_name=['Mr.', 'Mrs.', 'Miss', 'Ms.', 'Dr.','DVM', 'DDS', 'PhD', 'MD', 'Jr.', 'III', 'II', 'IV']
    first_name_col=[]
    last_name_col=[]
    
    #loop through the names in name column
    for name in name_col:
        #Split name into list of words
        name_lst = name.split(' ')
        #For list with more than two words, filter out the words found in 'not_name' list
        if len(name_lst)>2:
            name_lst= [n for n in name_lst if n not in not_name]
            #If list still contain more than 2 words after filtering, raise exception to alert data engineer.
            #Might be new salutation or title or profession e.g. CFA, CA etc
            if len(name_lst)>2:
                raise Exception(f"{name_lst} contain more than two words")
        
        #Append first and last name to their respective list
        first_name_col.append(name_lst[0])
        last_name_col.append(name_lst[1])
            
    return first_name_col, last_name_col



In [4]:
df3['first_name'], df3['last_name']=split_name(df3['name'])

## Requirement 3: Remove any zeros prepended to the price field

In [5]:
#Converting the price to float would automatically remove any unnecessary leading zeros
df3['price']=df3['price'].astype(float)

## Requirement 4: Create a new field named above_100, which is true if the price is strictly greater than 100

In [6]:
df3['above_100']=df3['price'].apply(lambda x: 'true' if x> 100 else None)

## Output result

In [7]:
df3= df3[['first_name', 'last_name', 'price', 'above_100']]
df3.to_csv(f'output/csv_files/{timestamp}.csv', index=False)